In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

from lift.utils import read_csv
from lift.backend.scenario import SingleScenario, ScenarioResult

In [5]:
def run_casestudy(casestudy_name: str, casestudy_df: pd.DataFrame):
    results = {scn: SingleScenario.from_series(scn_series).simulate() for scn, scn_series in casestudy_df.iterrows()}

    def costs4plotting(result: ScenarioResult) -> np.ndarray:
        costs = np.zeros(2 * (result.period_eco + 1))
        costs[2::2] = result.opex_dis[:-1]
        costs[1::2] = result.capex_dis
        return np.cumsum(costs)

    df_time = pd.DataFrame(data={k: costs4plotting(v) for k, v in results.items()}) * 1e-6
    df_time["year"] = np.repeat(np.arange(len(df_time) // 2), 2)
    df_time = df_time[["year"] + [col for col in df_time.columns if col != "year"]]
    df_time.to_csv(f"{casestudy_name}_data_time.csv", index=False)

    df_kpis = pd.DataFrame(
        {
            k: {
                "opex": v.opex_dis.sum() * 1e-6,
                "capex": v.capex_dis.sum() * 1e-6,
                "gamma_sc": v.self_consumption,
                "gamma_ss": v.self_sufficiency,
                "gamma_ch": v.home_charging_fraction,
                "co2": v.totem.sum() * 1e-6,
            }
            for k, v in results.items()
        },
        dtype=float,
    ).T
    df_kpis.index.name = "scenario"
    df_kpis[["gamma_sc", "gamma_ss", "gamma_ch"]] = df_kpis[["gamma_sc", "gamma_ss", "gamma_ch"]].fillna(1.1)
    df_kpis.to_csv(f"{casestudy_name}_data_total.csv")

In [6]:
df = read_csv(Path("case_a.csv"), transpose=True)
run_casestudy("A", df)
df = read_csv(Path("case_b.csv"), transpose=True)
run_casestudy("B", df)
df = read_csv(Path("case_c.csv"), transpose=True)
run_casestudy("C", df)

Scenario simulation time: 0.0415 s
Scenario simulation time: 0.5964 s
Scenario simulation time: 0.7899 s
Scenario simulation time: 0.0485 s
Scenario simulation time: 0.9044 s
Scenario simulation time: 0.9553 s
Scenario simulation time: 0.0335 s
Scenario simulation time: 0.2213 s
Scenario simulation time: 0.2381 s
